In [5]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, regexp_replace,round
from pyspark.sql.functions import expr

# Set the classpath to include the JAR file
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/jovyan/spark-streaming-kafka-0-8-assembly_2.11-2.0.0-preview.jar pyspark-shell'

# Initialize a SparkSession
spark = SparkSession.builder.appName("Kafka Message Processing").config("spark.driver.allowMultipleContexts", "true").config("hive.metastore.uris", "thrift://hive-metastore:9083").config("spark.sql.warehouse.dir", "hdfs://namenode:9000/hive").enableHiveSupport().getOrCreate()
sqlContext = SQLContext(spark)

# Tạo bảng Hive nếu chưa tồn tại
sqlContext.sql("""
    CREATE TABLE IF NOT EXISTS WeatherData (
        time int,
        day int,
        month int,
        temperature DOUBLE,
        feelslike DOUBLE,
        wind DOUBLE,
        cloud DOUBLE,
        rain DOUBLE,
        pressure DOUBLE,
        weather STRING
    )
    USING HIVE
""")

# Đọc dữ liệu từ tệp CSV
data = spark.read.format("libsvm").option("delimiter", "\t").csv("data/*.csv", header=True, inferSchema=True)

data = data.withColumn("time", expr("substring(time, 1, length(temperature) - 3)"))
data = data.withColumn("time", data["time"].cast("int"))
data = data.withColumn("day", data["day"].cast("int"))
data = data.withColumn("month", data["month"].cast("int"))

data = data.withColumn("temperature", expr("substring(temperature, 1, length(temperature) - 2)"))
data = data.withColumn("temperature", data["temperature"].cast("double"))

data = data.withColumn("forecast", expr("substring(forecast, 1, length(forecast) - 2)"))
data = data.withColumn("forecast", data["forecast"].cast("double"))
data = data.withColumnRenamed("forecast", "feelslike")

data = data.withColumn("wind", expr("substring(wind, 1, length(wind) - 4)"))
data = data.withColumn("wind", expr("wind / 3.6").cast("double"))
data = data.withColumn("wind", round(data["wind"], 2))


data = data.withColumn("cloud", expr("substring(cloud, 1, length(cloud) - 1)"))
data = data.withColumn("cloud", data["cloud"].cast("double"))

data = data.withColumn("rain", expr("substring(rain, 1, length(rain) - 2)"))
data = data.withColumn("rain", data["rain"].cast("double"))

data = data.withColumn("pressure", regexp_replace("pressure", "mb", ""))
data = data.withColumn("pressure", data["pressure"].cast("double"))


print(data.count())

data.show()
# Lưu dữ liệu vào bảng Hive
data.write.mode("overwrite").insertInto("WeatherData")
#t = sqlContext.sql("""
#    SELECT * FROM WeatherData LIMIT 100
#""")
#t.show()
# Đóng phiên Spark
spark.stop()


35744
+----+---+-----+-----------+---------+----+-----+----+--------+-------------+
|time|day|month|temperature|feelslike|wind|cloud|rain|pressure|      weather|
+----+---+-----+-----------+---------+----+-----+----+--------+-------------+
|   0|  1|    1|       24.0|     26.0|2.22|  4.0| 0.0|  1012.0|        Clear|
|   3|  1|    1|       24.0|     27.0|2.22|  4.0| 0.0|  1011.0|        Clear|
|   6|  1|    1|       24.0|     26.0|2.22|  7.0| 0.0|  1012.0|        Clear|
|   9|  1|    1|       28.0|     31.0|3.06|  6.0| 0.0|  1012.0|        Clear|
|  12|  1|    1|       32.0|     35.0|2.78| 53.0| 0.0|  1010.0|Partly cloudy|
|  15|  1|    1|       32.0|     36.0|0.28| 79.0| 0.0|  1009.0|       Cloudy|
|  18|  1|    1|       27.0|     30.0|1.39| 43.0| 0.0|  1010.0|Partly cloudy|
|  21|  1|    1|       26.0|     28.0|2.22| 29.0| 0.0|  1011.0|Partly cloudy|
|   0|  2|    1|       25.0|     27.0|2.78| 12.0| 0.0|  1011.0|        Clear|
|   3|  2|    1|       24.0|     26.0|2.22| 17.0| 0.0|  10